<h1>CNN-H1 3-26-2016</h1>

<strong>Abstract</strong>
Setting up the framework to train and test the MM-DFR. Implementing the CNN-H1 using NN2 described in the paper: http://arxiv.org/pdf/1509.00244v1.pdf

<strong>Improvements</strong>
<ul>
<li>Take the people with the highest number of images for validation (top 900)</li>
<li>Greyscale images - focus on the edges, features rather than colour</li>
<li>Prevent overfitting by horizontally flipping images</li>
<li>Normalize images to 230x230 (similar to the paper)</li>
</ul>

In [1]:
%load_ext autoreload

In [ ]:
%autoreload 2
%matplotlib inline

import os
import fnmatch
import numpy as np

from skimage import io
from skimage.color import rgb2grey
from skimage.transform import resize

import sklearn.metrics as metrics

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D, AveragePooling2D
from keras.optimizers import SGD
from keras.utils import np_utils
from keras.callbacks import EarlyStopping
from keras.models import model_from_json

np.random.seed(123456)

In [3]:
data_path = '../data/lfw_cropped'

img_rows, img_cols = 230, 230
train_size_percent = .85
validation_split = .15

<h2>Loading Files</h2>

In [4]:
def get_face_to_file_path_dict():
    face_to_file_paths_dict = {}
    
    for root, dirnames, filenames in os.walk(data_path):
        for dirname in dirnames:
            if dirname not in face_to_file_paths_dict:
                face_to_file_paths_dict[dirname] = []
            directory_path = os.path.join(data_path, dirname)
            for filename in os.listdir(directory_path):
                if filename.endswith(".jpg"):
                    face_to_file_paths_dict[dirname].append(os.path.join(directory_path, filename))
                            
    return face_to_file_paths_dict

In [5]:
def get_face_to_file_paths_descending_list(face_to_file_paths_dict):
    return sorted(face_to_file_paths_dict.items(), key=lambda x: len(x[1]), reverse=True)

In [6]:
face_to_file_paths_dict = get_face_to_file_path_dict()

In [7]:
face_to_file_paths_list = get_face_to_file_paths_descending_list(face_to_file_paths_dict)[:900]

<h2>Data Pre-Processing</h2>

In [8]:
def image_read(f):
    return resize(rgb2grey(io.imread(f)), (img_rows, img_cols))

In [9]:
def reflection(image):
    return np.array([list(reversed(row)) for row in image])

In [10]:
images_by_class = [[image_read(f) for f in x[1]] for x in face_to_file_paths_list]

In [11]:
for i in range(len(images_by_class)):
    images_by_class[i] += [reflection(image) for image in images_by_class[i]]

In [12]:
X_train = np.array([image for images in images_by_class for image in images])
X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)

In [13]:
y_train = np.array([images[0] for images in enumerate(images_by_class) for image in images[1]])
Y_train = np_utils.to_categorical(y_train, len(images_by_class))

<h2> Training and Validation</h2>

In [14]:
def NN2(input_shape, nb_classes, nb_fc6):
    model = Sequential()
    
    # Layer 1
    model.add(Convolution2D(64, 3, 3, activation='relu', input_shape=input_shape))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    # Layer 2
    model.add(Convolution2D(64, 3, 3, activation='relu'))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    # Layer 3
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    #Layer 4
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    #Layer 5
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3))
    model.add(ZeroPadding2D((1,1)))
    model.add(AveragePooling2D((2,2), strides=(2,2)))

    model.add(Flatten())
    model.add(Dropout(0.4))
    model.add(Dense(nb_fc6))
    model.add(Dense(nb_classes, activation='softmax'))
   
    return model

In [15]:
input_shape = (1, 230, 230)
nb_classes = len(images_by_class)
nb_fc6 = 512

In [ ]:
model = NN2(input_shape, nb_classes, nb_fc6)
model.compile(loss='categorical_crossentropy', optimizer='sgd')
model.fit(X_train, Y_train, batch_size=1, nb_epoch=10, 
        show_accuracy=True, verbose=1, shuffle=True, validation_split=.15)

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer=SGD(lr=0.001))
early_stopping = EarlyStopping(monitor='val_loss', patience=3, mode='min')
model.fit(X_train, Y_train, batch_size=1, nb_epoch=50, early_stopping=early_stopping, 
        show_accuracy=True, verbose=1, shuffle=True, validation_split=.15)

In [ ]:
json_string = model.to_json()
open('models/CNN-H1 .json', 'w').write(json_string)
model.save_weights('models/CNN-H1.h5')